# CAT Staffing

In [1]:
import pandas as pd
from gurobipy import *

### Data Input

In [2]:
initialProjects = pd.read_csv('InitialProjects.csv')

In [3]:
initialPeople = pd.read_csv('InitialPeople.csv')

In [4]:
#here is probably where you also set up a way to get additional data fed in

### Sets

In [5]:
people = list(initialPeople.person.unique())

In [6]:
projects = list(initialProjects.PROJECT_NAME.unique())

In [7]:
time = [i for i in range(1,13)]

In [8]:
initialPeople.head()

,person,skillset,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Scott,"[[S1,3],[S2,1],[S3,2]]",NaN,NaN,NaN,NaN,NaN,NaN
1,Yuriy,"[[S4,2],[S5,2],[S6,1],[S7,2]]",NaN,NaN,NaN,NaN,NaN,NaN
2,Jenny,"[[S1,1],[S2,1],[S8,2]]",NaN,NaN,NaN,NaN,NaN,NaN
3,Grace,"[[S2,1],[S2,2],[S7,1]]",NaN,NaN,NaN,NaN,NaN,NaN
4,Evan,"[[S16,3],[S3,3],[S4,2]]",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
initialProjects.head()

,LABEL,PROJECT_NAME,DEADLINE,HAS_DEADLINE?,RELATED_TO_DELIVERABLE,NO_PROJECT_DEPENDENCIES?,L/G?,HAVE_ALL_DATA?,COMPLETION_GOAL,START_DATE,END_DATE,Tools/Skills,PROJECT_STATUS,Dependencies?,PEOPLE_WORKING,Notes,SORT_SCORE,EST_HOURS_REQ
0,DASA-CE,ARRG Slide Stats,3/1/2018,1,1,1,1,1,43160,43132,NaN,"Python/R, SQL",Active,NaN,"Yuriy, Scott",NaN,9,10
1,DASA-CE,Price Change Dashboard/Analysis,3/1/2018,1,1,1,1,1,43160,43160,NaN,"Python/R, Qlik, SQL",Active,NaN,Whole Team,NaN,9,10
2,OSMIS,R/Python SQL Integration,NaN,0,1,1,1,1,NaN,43132,NaN,"R/Python, SQL",Active,NaN,Yuriy,NaN,6,10
3,OSMIS,CF Business Rule Review,NaN,0,1,1,1,1,43160,43132,NaN,Python/R,Active,NaN,"Grace, Yuriy",FC/AR -- Grace ; Density -- Yuriy,6,10
4,OSMIS,ATR Demand Forecast,NaN,0,1,1,1,1,43252,43101,NaN,"Python/R, Qlik Sense, SQL, Forecasting",Active,NaN,"Scott, Evan, Jenny",Waiting on quarterly QVD's,6,10


### Data Pre-Processing

In [10]:
#requirements
#availability of people - this could be calculated as number of work days over the planning horizon

#skillSupply
skillSupply = dict(zip(initialPeople.person,initialPeople.skillset))  

#number of hours in month t
hrsAvailable = {1:160,2:160,3:160,4:160,5:160,6:160,7:160,8:160,9:160,10:160,11:160,12:160}

#estimated hours required by project
estHoursReq = dict(zip(initialProjects.PROJECT_NAME,initialProjects.EST_HOURS_REQ))

#skillDemand
skillDemand = dict(zip(initialProjects.PROJECT_NAME,initialProjects['Tools/Skills']))

### Model

In [11]:
m = Model('CAT Staff')

GurobiError: License expired 2018-05-04 - license file 'C:\Users\evan.lynch\gurobi.lic'

#### Decision Variables

In [ ]:
#Whether to staff someone on a project
StaffDecision = m.addVars(people,projects,time,vtype=GRB.CONTINUOUS,name='StaffingDecision')
Overstaffing = m.addVars(people,time,vtype=GRB.CONTINUOUS,name='OverstaffPercent')
Understaffing = m.addVars(people,time,vtype=GRB.CONTINUOUS,name='UnderstaffPercent')

#### Objective

In [ ]:
#minimize total understaffing
m.setObjective(quicksum(Understaffing[i,t] for i in people for t in time),GRB.MINIMIZE)

#### Constraints

In [ ]:
#define staffing
StaffingEquilibrium = m.addConstrs((quicksum(StaffDecision[i,j,t]*estHoursReq[j]/hrsAvailable[t] for j in projects)
                                    == 1 + Overstaffing[i,t] - Understaffing[i,t] for i in people for t in time),"StaffingEquilibrium")

#all projects must be assigned
AssignAllProjects = m.addConstrs((quicksum(StaffDecision[i,j,t] for i in people for t in time) 
                               >=1 for j in projects),"AssignAllTasks")


StdLT = 11
startDate = m.addVars(projects,vtype=GRB.INTEGER,name='StartDate')
Deadline = 12
StdStartDate = 1

#ideas for constraints
#person needs to have skill in order to work on project
#project must be scheduled with enough lead time to finish it before the expected end date
#StartDate = m.addConstrs((startDate[j] == (-1*(Deadline - StdStartDate)*quicksum(StaffDecision[i,j,t] for i in people for t in time))/StdLT for j in projects),"StartDate")

#StartDate = m.addConstrs((StdLT/quicksum(StaffDecision[i,j,t] for i in people for t in time) - startDate[j] == Deadline[j] - StdStartDate[j] for j in projects),"StartDate")
#if staffed on a project for the first time, continue it until the project is supposed to be complete
#maybe have a manual way to enter in here that a person can't be staffed on a particular project (if we really need that)
#give priority to projects that have a higher priority score
#if project is inactive, don't staff it

#### Run

In [ ]:
m.optimize()

In [ ]:
for j in projects:
    print(startDate[j].x)

In [ ]:
for i in people:
    for j in projects:
        for t in time:
            print((i,j,t),StaffDecision[i,j,t].x)